#### Source:

Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.

#### Data Set Information:

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.



#### Attribute Information:

- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.

In [31]:
# libraries

import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 100

In [2]:
df = pd.read_excel("Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


- clearly 2 columns has missing values

In [45]:
df.Description.isna().sum(), df.CustomerID.isna().sum()

(1454, 135080)

- InvoiceNo normally should have numbers, but on examination we found that there are invoices starting with 'C', these are the Cancellations/Returns

- Lets make a subset of the data, which is without Cancellations/Return

In [46]:
#df_without_can = df[~df.InvoiceNo.str.startwith("C")]

df_without_can = df.loc[~df.InvoiceNo.str.startswith('C', na=False)]


In [47]:
df.shape[0]-df_without_can.shape[0]

9288

In [48]:
df.shape, df_without_can.shape

((541909, 8), (532621, 8))

In [49]:
df_without_can.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532621 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      532621 non-null object
StockCode      532621 non-null object
Description    531167 non-null object
Quantity       532621 non-null int64
InvoiceDate    532621 non-null datetime64[ns]
UnitPrice      532621 non-null float64
CustomerID     397924 non-null float64
Country        532621 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.6+ MB


- There are still missing data, if the Description is missing we can't do Market Basket Analysis on them, so lets drop them

In [50]:
df_without_can.Description = df_without_can.Description.str.strip()
df_without_can_and_nan = df_without_can.loc[df_without_can.Description.notnull()]
df_without_can_and_nan.shape

(531166, 8)

In [51]:
df_without_can_and_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531166 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      531166 non-null object
StockCode      531166 non-null object
Description    531166 non-null object
Quantity       531166 non-null int64
InvoiceDate    531166 non-null datetime64[ns]
UnitPrice      531166 non-null float64
CustomerID     397924 non-null float64
Country        531166 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.5+ MB


In [94]:
len(df_without_can_and_nan[df_without_can_and_nan.Quantity == 0])

0

In [52]:
df_without_can_and_nan.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


##### Clean the Description column


In [53]:
df_without_can_and_nan.Description.unique()

array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ..., 'lost',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], dtype=object)

 There are a lot of junks in this column :
 
 - lost
 - lost in space
 - POSSIBLE DAMAGES OR LOST?
 - lost??
 - ?lost
 - Damaged
 - wet damaged
 - samples/damages
 - POSSIBLE DAMAGES OR LOST?
 - damages/dotcom?
 - damages?
 - missing
 - ?missing
 - wrongly marked. 23343 in box
 - stock creditted wrongly
 - wrongly sold sets
 - sold with wrong barcode
 - smashed ......... etc etc
 

We can create a list of all such items and filter them out of the data, or we can just make a short list of root words and eliminate all the data that contains them.

In [60]:
# Convert the entire column to lowercase and remove unnecessary spaces : for consistency

df_without_can_and_nan.Description = df_without_can_and_nan.Description.str.lower()


In [66]:
# Create a list of all unique Description and write to a excel file for further examination

unique_desc_list = df_without_can_and_nan.Description.unique()
unique_desc_list.sort()

unique_desc = pd.DataFrame(unique_desc_list)
writer = pd.ExcelWriter('items_100.xlsx', engine='xlsxwriter')
unique_desc.to_excel(writer,sheet_name='unique list of items',index=False)
writer.save()

print(len(unique_desc_list))

4177


In [116]:
# Creating the list junk words (got all the analysis of such words done in excel)

unique_desc_cleaned = pd.read_excel("items.xlsx")
unique_desc_cleaned.head(2)

,items,junk,item_class
0,*boombox ipod classic,NaN,NaN
1,*usb office mirror ball,NaN,NaN


- We found that there are many items in the invoice for Gift wraps too, we marked them in 'class' column and we marked 'Yes' in 'junk' column for the items that needs to be dropped

In [117]:
# Drop these items from the our data

drop_list = unique_desc_cleaned[~((unique_desc_cleaned.junk == 'Yes') | (unique_desc_cleaned.item_class == 'Wrap'))]['items']
print("Total number of unique items to be considered :",len(drop_list))

new_data = df_without_can_and_nan[df_without_can_and_nan.Description.isin(drop_list)]
new_data.shape


Total number of unique items to be considered : 3943


(523175, 8)

- Lets check the Quantity column for values less than 0 (these may be returned items)

In [118]:
new_data[new_data.Quantity <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
151033,549527,21620,mystery! only ever imported 1800,-1479,2011-04-08 16:03:00,0.0,NaN,United Kingdom
323872,565370,82494L,crushed ctn,-33,2011-09-02 15:04:00,0.0,NaN,United Kingdom
381679,569878,90057,crushed boxes,-380,2011-10-06 15:12:00,0.0,NaN,United Kingdom
418065,572686,23118,breakages,-30,2011-10-25 14:03:00,0.0,NaN,United Kingdom


In [119]:
# Dropping these data from our dataset

data = new_data.loc[~new_data.Quantity <= 0]
data.shape

(523171, 8)

In [120]:
# Change the Date format - remove the time stamp

data.InvoiceDate = data["InvoiceDate"].dt.strftime("%m-%d-%y")
data.head()

/Users/kumarchk/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,6,12-01-10,2.55,17850.0,United Kingdom
1,536365,71053,white metal lantern,6,12-01-10,3.39,17850.0,United Kingdom
2,536365,84406B,cream cupid hearts coat hanger,8,12-01-10,2.75,17850.0,United Kingdom
3,536365,84029G,knitted union flag hot water bottle,6,12-01-10,3.39,17850.0,United Kingdom
4,536365,84029E,red woolly hottie white heart.,6,12-01-10,3.39,17850.0,United Kingdom
